# Use SAP AI Core to Train a Pose Estimation Nueral Network


## Step 1

Create a connection with AI Core. Use the `aicore_creds.json` provided to you and place it in the project working directory.

In [1]:
import os
import json

with open('aicore_creds.json') as f:
    ac_creds = json.load(f)
    
os.environ["AICORE_AUTH_URL"] = f"{ac_creds['url']}/oauth/token"
os.environ["AICORE_BASE_URL"] = ac_creds['serviceurls']['AI_API_URL']
os.environ["AICORE_CLIENT_ID"] = ac_creds['clientid']
os.environ["AICORE_CLIENT_SECRET"] = ac_creds['clientsecret']
os.environ["AICORE_RESOURCE_GROUP"] = "default"

In [21]:
# Load Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client(
    base_url = os.environ["AICORE_BASE_URL"] + "/v2", # The present SAP AI Core API version is 2
    auth_url=  os.environ["AICORE_AUTH_URL"], 
    client_id = os.environ["AICORE_CLIENT_ID"],
    client_secret = os.environ["AICORE_CLIENT_SECRET"]
)
# no output is expected

## Step 2

Create a Docker account and generate a personal access token. Refer to [this](https://developers.sap.com/tutorials/ai-core-code.html#cf7b33ab-c455-47ee-a812-33a1ff587cf0) tutorial for steps. Also install Docker Desktop [here](https://www.docker.com/products/docker-desktop/).

## Step 3

Write the training code. You can find this is `train_movenet.py`, `model_utils.py`, and `preprocessing_utils.py`.

## Step 4

Define a `Dockerfile` and ensure it successfully builds using the make target `deploy-container-service-local`. Then connect Docker to your account via `docker login docker.io`. Now upload the image to the cloud using the make target `push-docker-image`.

## Step 5

Store your Docker credentials in SAP AI Core as a Docker registry secret.

In [28]:
with open('docker_creds.json') as f:
    docker_creds = json.load(f)

In [29]:
response = ai_core_client.docker_registry_secrets.create(
    name = "docker-secret",
    data = {
        ".dockerconfigjson": f'{{\"auths\":{{\"docker.io/amanichopra/aicore-train:tagname\":{{\"username\":\"amanichopra\",\"password\":\"{docker_creds["pat"]}\"}}}}}}'
    }
)

print(response.__dict__)

{'message': 'Secret exists'}


## Step 6

You need to onboard your Github Repo to AI Core.

In [19]:
with open('gh_creds.json') as f:
    gh_creds = json.load(f)

In [21]:
# on-board new GitHub repository
ai_core_client.repositories.create(
    name = "github-repo",
    url = "https://github.com/amanichopra/sap-aicore-train.git",
    username = "amanichopra",
    password = gh_creds['pat']
)

In [22]:
# check on-boarding status
response = ai_core_client.repositories.query()
#
for repository in response.resources:
#    print('Name:', repository.name)
    print('URL:', repository.url)
    print('Status:', repository.status)

URL: https://github.com/amanichopra/sap-aicore-train.git
Status: RepositoryStatus.COMPLETED


## Step 7

Create an application to sync the training workflow with AI Core.

In [7]:
ai_core_client.applications.create(
    application_name = "training-app",
    repository_url = "https://github.com/amanichopra/sap-aicore-train.git",
    path = "./",
    revision = "HEAD"
)

In [8]:
response = ai_core_client.applications.query()

for app in response.resources:
    print(app.__dict__)

{'path': './', 'revision': 'HEAD', 'repository_url': 'https://github.com/amanichopra/sap-aicore-train.git', 'application_name': 'training-app'}


## Step 8

Store S3 object store secrets in AI Core.

In [25]:
with open('s3_creds.json') as f:
    s3_creds = json.load(f)

In [26]:
# Create object Store secret
response = ai_core_client.object_store_secrets.create(
    name = "s3-secret", # identifier for this secret within your SAP AI Core
    path_prefix = "data", # path that we want to limit restrict this secret access to
    type = "S3",
    data = { # Dictionary of credentials of AWS
        "AWS_ACCESS_KEY_ID": s3_creds['access_key_id'],
        "AWS_SECRET_ACCESS_KEY": s3_creds['secret_access_key']
    },
    bucket = "hcp-c1a2d095-b523-400a-bf19-94eda5e8d109", # Edit this
    region = "eu-central-1", # Edit this
    endpoint = "s3-eu-central-1.amazonaws.com", # Edit this
    resource_group = "default" # object store secret are restricted within this resource group. you may change this when creating secret for another resource group.
)
print(response.__dict__)

{'message': 'Secret exists'}


In [27]:
# Create object Store secret
response = ai_core_client.object_store_secrets.create(
    name = "default", # identifier for this secret within your SAP AI Core
    path_prefix = "model", # path that we want to limit restrict this secret access to
    type = "S3",
    data = { # Dictionary of credentials of AWS
        "AWS_ACCESS_KEY_ID": s3_creds['access_key_id'],
        "AWS_SECRET_ACCESS_KEY": s3_creds['secret_access_key']
    },
    bucket = "hcp-c1a2d095-b523-400a-bf19-94eda5e8d109", # Edit this
    region = "eu-central-1", # Edit this
    endpoint = "s3-eu-central-1.amazonaws.com", # Edit this
    resource_group = "default" # object store secret are restricted within this resource group. you may change this when creating secret for another resource group.
)
print(response.__dict__)

{'message': 'Secret exists'}


## Step 9

Register the dataset.

In [16]:
# Create Artifact
from ai_api_client_sdk.models.artifact import Artifact

response = ai_core_client.artifact.create(
    name = "pose_training_data", # Custom Non-unqiue identifier
    kind = Artifact.Kind.DATASET,
    url = "ai://s3-secret/", #
    scenario_id = "train-demo",
    description = "Pose embedding data",
    resource_group = "default" # required to restrict object store secret usage within a resource group
)

print(response.__dict__)

{'id': '3611f615-d041-4db0-8e1a-70e2784bd6bd', 'message': 'Artifact acknowledged', 'url': 'ai://s3-secret/'}


## Step 10

Define your workflow using [Argo](https://argo-workflows.readthedocs.io/en/latest/workflow-templates/) workflow templates. This is defined at `train_pipeline.yaml`.


## Step 11

Check the sync status of your workflow. You should see a successful sync in the response.

In [9]:
# Get application status
response = ai_core_client.applications.get_status(application_name='training-app')

print(response.message)
for sync_status in response.sync_ressources_status:
    print(sync_status.__dict__)

successfully synced (all tasks run)
{'name': 'aicore-train-pipeline', 'kind': 'WorkflowTemplate', 'status': 'Synced', 'message': 'workflowtemplate.argoproj.io/aicore-train-pipeline created'}


## Step 12

Ensure you can see the executable defined in the workflow template for this scenario.

In [12]:
# List available executables
response = ai_core_client.executable.query(
    scenario_id = "train-demo", resource_group='default'
)

for executable in response.resources:
    print(executable.__dict__)


{'id': 'aicore-train-pipeline', 'scenario_id': 'train-demo', 'version_id': '2.0', 'name': 'training', 'description': 'Train pose detection model with live data', 'deployable': False, 'parameters': [<ai_api_client_sdk.models.parameter.Parameter object at 0x124a2f890>, <ai_api_client_sdk.models.parameter.Parameter object at 0x124a2ed50>, <ai_api_client_sdk.models.parameter.Parameter object at 0x124a2f770>, <ai_api_client_sdk.models.parameter.Parameter object at 0x124a2f530>, <ai_api_client_sdk.models.parameter.Parameter object at 0x124a2ff20>, <ai_api_client_sdk.models.parameter.Parameter object at 0x124a0ac90>], 'input_artifacts': [<ai_api_client_sdk.models.input_artifact.InputArtifact object at 0x1247a63c0>], 'output_artifacts': [<ai_api_client_sdk.models.output_artifact.OutputArtifact object at 0x123d7c0e0>], 'labels': None, 'created_at': datetime.datetime(2024, 11, 26, 22, 36, 30, tzinfo=datetime.timezone.utc), 'modified_at': datetime.datetime(2024, 11, 26, 22, 36, 30, tzinfo=datetim

## Step 13

Create a configuration for the training workflow.

In [28]:
# Create configuration
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.base_models import KeyValue

response = ai_core_client.configuration.create(
    name = "pose-detection-training-configuration",
    scenario_id = "train-demo",
    executable_id = "aicore-train-pipeline",
    resource_group = "default",
    parameter_bindings=[ParameterBinding(KeyValue(key='DL', value='5'))]
)
print(response.__dict__)


TypeError: KeyValue.__init__() missing 1 required positional argument: 'value'

In [ ]:
# Create and start execution
response = ai_core_client.execution.create(
    configuration_id = '<YOUR_CONFIGURATION_ID>',
    resource_group = 'default'
)

response.__dict__

In [ ]:
# Get execution status from SAP AI Core
# execute this multiple times in interval of 30 seconds
response = ai_core_client.execution.get(
    execution_id = '<YOUR_EXECUTION_ID', # Change this
    resource_group = 'default'
)

response.__dict__


- should end up with **Status: DEAD**

## Step 11 Look for error logs in execution


In [ ]:
# get execution logs
response = ai_core_client.execution.query_logs(
    execution_id = '<YOUR_EXECUTION_ID>',
    resource_group = 'default',
    start = datetime(1990, 1, 1) # Optional, required when execution is older than one hour
)

for log in response.data.result:
    print(log.__dict__)


## Step 12: Fix and update code

*Refer step 12 of the tutorial: https://developers.sap.com/tutorials/ai-core-code.html#d3cdbc12-922d-48b2-a197-44f0d8fbd867*

## Step 13: Update your AI workflow

*Refer step 13 of the tutorial: https://developers.sap.com/tutorials/ai-core-code.html#3fedf413-00e5-4554-85bc-06130758678a*

## Step 14: Create Execution

In [ ]:
# Create execution
response = ai_core_client.executable.query(
    scenario_id = "learning-code", resource_group='default'
)

for executable in response.resources:
    print(executable.__dict__)


In [ ]:
# Get execution status from SAP AI Core
# execute this multiple times in interval of 30 seconds
response = ai_core_client.execution.get(
    execution_id = '<YOUR_EXECUTION_ID', # Change this
    resource_group = 'default'
)

response.__dict__


- should show **Status: COMPLETED**

In [ ]:
# Get Execution Logs
response = ai_core_client.execution.query_logs(
    execution_id = '<YOUR_EXECUTION_ID>',
    resource_group = 'default'
)

for log in response.data.result:
    print(log.__dict__)
